# RXMCによる[y=ax+b]のfittingとBayes推定
### 行うこと
+ RXMC
+ RXMCの結果からBayes推定(事後確率分布の生成)
 --> まずはここまで

### 00.事前準備
+ 現在位置の確認
+ ディレクトリの生成


In [ ]:
####### 操作設定 #######
import os
from jnbayes.libs.etc_sub import make_directory as mk_dir
print("###### start programs ######")
print("NOW CWD IS", os.getcwd())

# 各種保存用ディレクトリ作成
print("######### make dirs #########")
mk_dir("./dt/")
mk_dir("./dt/np/")
mk_dir("./dt/json/")
mk_dir("./pic/")
mk_dir("./bayes_log/")




### 01.データ生成
+ npyファイルの生成、保存

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

n = 100
sigma_R = 0.5#ノイズの標準偏差
x = np.arange(n)/10
a = 1.9
b = 5.5

y = a*x + b + np.random.normal(loc=0, scale=sigma_R,size=n)


data = [x,y]
plt.scatter(x,y)
plt.show()
np.save("dt/np/liner.npy",data)
plt.close()


### 02.パラメータ設定
+ ファイルパスやRXMCのループ数などの設定
+ ハイパーパラメータの設定

In [ ]:
from jnbayes.libs.rxmc_cls import Rxmc_ctrl
from jnbayes.libs.etc_sub import read_xy_slice_data
import json

liner_json_path = "./dt/json/liner_test.json"

liner_json = {
    "dpar": {
        "model_name":"ax1b",        # モデル名
        "dt_file": "dt/np/",        # 元データのパス
        "log_file": "bayes_log/",   # おまじない、bayesのログ保管パス
        "pic_file": "pic/",         # おまじない、画像出力パス
        "dt_name": "liner",         # データ名(拡張子なし)
        "dt_extension": ".npy",     # データ拡張子
        "log_par_rxmc_loops": 1000, # おまじない、RXMCログ保存数
        "bayes_Efree_par_rxmc_loops": 1000 #おまじない、bayes自由エネルギー抽出数
},
    "rpar": {
        "cycle": 5000,              # RXMCサイクル数
        "num_temp": 70,             # レプリカ数
        "proportion": 1.3,          # 逆温度等比級数
        "burn_in_length": 1000      # 焼きなまし回数
    },
    "fpar": {
        "convolve":{"TF":False},    # おまじない、特殊fitting用
        "xlim":[None,None],         # おまじない、fittingのデータ点を切れる
        "noise": {
            "sigmaR": 0.25,         # 推定ノイズ、大体半分の値を入力
            "sigmaE": 1.0           # おまじない、使わない
        },
        "BG": {
            "model": "nothing"      # BGの値、
        },
        "peak": {
            "01": {                 # 第一関数セット、複数セットができる
                "model": "liner",   # 関数名、
                "par": {
                    "init": [
                        2,          # aの初期値
                        5.4         # bの初期値
                    ]
                },
                "hyper": {
                    "step": [
                        7,          # aのC値
                        7           # bのC値
                    ],
                    "alpha": [
                        40000.0,    # aのα値
                        1000.0      # bのα値
                    ],
                    "dd": [
                        0.5,        # aのd値
                        0.5         # bのd値
                    ]
                },
                "limit": {
                    "rng_min": [
                        0,          # aの推定最小値
                        0           # bの推定最小値
                    ],
                    "rng_max": [
                        10,         # aの推定最大値
                        10          # bの推定最大値
                    ]
                }
            }
        }
    }
}

with open(liner_json_path,"w") as fout:
    json.dump(liner_json, fout, indent=4)


#### 上で書いたjsonを専用のクラスで読み込み
rx_cls = Rxmc_ctrl(liner_json_path)
print("class read OK!")

### XYデータ(npyファイル)を読み込み、クラスに登録
read_xy_slice_data(rx_cls, check=False)
print("XY data read OK!")

### RXMCのログ保存ディレクトリを作成
mk_dir(rx_cls.log_dt_name)

### クラスにXYデータが登録できたか確認
plt.scatter(rx_cls.X,rx_cls.Y)
plt.show()
plt.close()


### 03.RXMC事前準備(rxmc_run.py->run_rxmc()前半)
+ RXMCのパラメータを読み込む
+ 初期値を代入、E(最小二乗誤差)の計算
+ レプリカの作成

In [ ]:
from jnbayes.libs.rxmc_sub import get_fitting_container
from jnbayes.libs.rxmc_sub import get_Y
from jnbayes.libs.rxmc_sub import get_Energy


######## read raw data
X = rx_cls.X
Y = rx_cls.Y

######## deploy param to meaningful parameters
dir_dict = rx_cls.df["dpar"]
func_dict = rx_cls.df["fpar"]
run_dict = rx_cls.df["rpar"]

######## read dict
_p_init_ = rx_cls.get_param("par","init")
_step_ = rx_cls.get_param("hyper","step")
_alpha_ = rx_cls.get_param("hyper","alpha")
_d_ = rx_cls.get_param("hyper","dd")
_p_min_ = rx_cls.get_param("limit","rng_min") 
_p_max_ = rx_cls.get_param("limit","rng_max")
log_length = rx_cls.df["dpar"]["log_par_rxmc_loops"]
cycle = run_dict["cycle"]
num_temp = run_dict["num_temp"]
sigmaE = func_dict["noise"]["sigmaE"]    
X_slice = rx_cls.xslice
######## get fitting container
fit_cont, BG_cont = get_fitting_container(func_dict)


######## calc dict
data_len = len(X)
par_num = len(_p_init_)
_p_min_tl_ = np.tile(_p_min_,(num_temp,1))
_p_max_tl_ = np.tile(_p_max_,(num_temp,1))
dt_log_name = rx_cls.logfile_header
if func_dict["convolve"]["TF"]:
    conv_TF = True
else:
    conv_TF = False

######## calc first ones
Yraw = Y.reshape(1,-1)
_p_now_ = np.tile(_p_init_,(num_temp,1))

Ytmp = get_Y(X,_p_now_,fit_cont, BG_cont, conv_TF)

_E_now_ = get_Energy(Yraw,Ytmp,data_len,sigmaE,X_slice)

######## cleate temp
temp, _stepsize_p_ = rx_cls.get_temp_step(par_num,_step_,_alpha_,_d_)


######## define for RXMC
p0_log = np.zeros((num_temp,par_num,log_length),dtype=np.int32)
E0_log = np.zeros((num_temp,log_length),dtype=np.int32)
_p_log_ = 1.0*p0_log
_E_log_ = 1.0*E0_log
_ratio_p_log_ = 0*p0_log
_ratio_exc_rep_log_ = 0*E0_log

### 04.結果の表示(03の)
+ 逆温度の範囲のプロット
+ stepsizeと逆温度のプロット

In [ ]:
### 逆温度のプロット
temp_range = np.array(range(len(temp)))
plt.plot(temp_range,temp)
plt.scatter(temp_range,temp,s=3)
plt.xlabel("number")
plt.ylabel("reverse temperature")
plt.yscale("log")
plt.title("reverse temperature")
plt.show()
plt.close()


### stepsizeと逆温度のプロット
for ipar in range(len(_stepsize_p_[0])):
    plt.plot(temp[1:-1],_stepsize_p_[1:-1,ipar])
    plt.scatter(temp[1:-1],_stepsize_p_[1:-1,ipar],s=3)
    plt.xlabel("rev. temp")
    plt.ylabel("stepsize")
    plt.xscale("log")
    plt.yscale("log")
    plt.title(f"param {ipar} stepsize")
    plt.show()
    plt.close()


### 05. RXMC開始(rxmc_run.py->run_rxmc()後半)
+ RXMC
+ ログファイルの記入


In [ ]:
import time
import gc
from jnbayes.libs.rxmc_sub import update_param
from jnbayes.libs.rxmc_sub import exc_replica
from jnbayes.libs.rxmc_sub import renew_E_log
from jnbayes.libs.rxmc_sub import renew_p_log


tm_start = time.time()



######## event loop <----- MOST IMPORTANT!!!
print("cycle =",cycle)
for icycl in range(cycle):
    
    ########## print progress of RXMC
    if icycl%100==0:
        tm_stop = time.time()
        time_str = str(round(100/(tm_stop-tm_start+(10e-5)),1)) + "Hz"
        print("event loop =",rx_cls.model_name, icycl,time_str)
        tm_start = tm_stop
    log_mod_cycl = icycl%log_length

    ########## update all parameters
    _p_now_, _E_now_ = update_param(icycl,log_length,_E_now_,data_len,sigmaE,X,Yraw,
                    _p_now_,_stepsize_p_,_ratio_p_log_,num_temp,
                    _p_min_tl_,_p_max_tl_,temp,fit_cont, BG_cont,X_slice,conv_TF)
    _p_log_[:,:,log_mod_cycl] = _p_now_

    ########## exchange replica
    exc_replica(icycl,_p_now_,_E_now_,temp,data_len,num_temp,log_mod_cycl,_ratio_exc_rep_log_)
    _E_log_[:,log_mod_cycl] = _E_now_

    ######## save log pars
    if (icycl+1)%log_length == 0:
        loop_num_str = str(int((icycl+1)/log_length)).zfill(3)
        tmp_arg = [num_temp,par_num,log_length]
        _p_log_ = renew_p_log(dt_log_name,loop_num_str,"_p_log_",_p_log_,tmp_arg)
        _E_log_ = renew_E_log(dt_log_name,loop_num_str,"_E_log_",_E_log_,tmp_arg)
        _ratio_p_log_ = renew_p_log(dt_log_name,loop_num_str,"_ratio_p_log_",_ratio_p_log_,tmp_arg)
        _ratio_exc_rep_log_ = renew_E_log(dt_log_name,loop_num_str,"_ratio_exc_rep_log_",_ratio_exc_rep_log_,tmp_arg)
        print("saved log par",log_length,"events")
        gc.collect() #### Garbage Collector cleans memory
print("event loop =",rx_cls.model_name,"F!!!!")

### 06.結果の表示(05の)
+ 全レプリカの全パラメータのログ
+ 全レプリカの最小二乗誤差のログ


In [ ]:
import glob

burn_in_length = run_dict["burn_in_length"]
logfile_header = rx_cls.logfile_header

#### パラメータログの描画関数
def plt_p_log(tmp_log,pic_name,par_num):
    np_nums = np.arange(len(tmp_log))
    plt.figure(figsize=(15,5))
    plt.title(pic_name)
    plt.subplot(1,2,1)
    plt.scatter(np_nums,tmp_log)
    plt.ylabel(f"par{par_num}")
    plt.xlabel("Iteration number")
    plt.subplot(1,2,2)
    plt.hist(tmp_log,bins=40)
    plt.xlabel(f"par{par_num}")
    plt.ylabel("Cnt")
    plt.show()
    plt.close()
    return

##### 全レプリカの全パラメータのログ
print("##############################")
print("全レプリカの全パラメータのログ")
print("##############################")
p_log_files = glob.glob(logfile_header+"_p_log_*.npy") # globでファイルサーチ
p_log_files = sorted(p_log_files)
p_log = np.empty((num_temp,par_num,0))
for i in range(len(p_log_files)):
    tfile = p_log_files[i]
    np_tmp = np.load(tfile,allow_pickle=True)
    p_log = np.append(p_log,np_tmp,axis=2)
for ipar in range(par_num):
    for irep in range(num_temp):
        tmp_p_log = p_log[irep][ipar][burn_in_length:]
        pic_name = f"par{ipar}, rep{irep}"
        plt_p_log(tmp_p_log,pic_name,ipar)



#### エネルギーログの描画関数
def plt_E_log(tmp_log,pic_name):
    np_nums = np.arange(len(tmp_log))
    plt.figure(figsize=(15,5))
    plt.title(pic_name)
    plt.subplot(1,2,1)
    plt.scatter(np_nums,tmp_log)
    plt.ylabel("E")
    plt.xlabel("Iteration number")
    plt.subplot(1,2,2)
    plt.hist(tmp_log,bins=40)
    plt.yscale("log")   
    plt.xlabel("E")
    plt.ylabel("Cnt")
    plt.show()
    plt.close()
    return

##### 全レプリカのエネルギーのログ
print("##############################")
print("全レプリカのエネルギーのログ")
print("##############################")
E_log_files = glob.glob(logfile_header+"_E_log_*.npy")
E_log_files = sorted(E_log_files)
E_log = np.empty((num_temp,0))
for ifile in range(len(E_log_files)):
    tfile = E_log_files[ifile]
    tmp = np.load(tfile,allow_pickle=True)
    E_log = np.append(E_log,tmp,axis=1)
for irep in range(num_temp):
    pic_name = f"rep{irep}"
    tmp_E_log = E_log[irep][burn_in_length:]
    plt_E_log(tmp_E_log,pic_name)




### 07.続き(06の)
+ パラメータ採択率 vs 逆温度
+ レプリカ交換率 vs 逆温度

In [ ]:

# パラメータ採択率 vs 逆温度
def plt_p_ratio(ipar,temp,ratio):
    par_str = "par"+str(ipar).zfill(2)
    plt.hlines(1.0,min(temp[1:]),max(temp[1:]),colors="gray")
    plt.hlines(0.0,min(temp[1:]),max(temp[1:]),colors="gray")
    plt.hlines(0.8,min(temp[1:]),max(temp[1:]),colors="gray",linestyle="dashed")
    plt.hlines(0.2,min(temp[1:]),max(temp[1:]),colors="gray",linestyle="dashed")
    plt.plot(temp[1:],ratio[1:])
    plt.scatter(temp[1:],ratio[1:],s=3)
    plt.title("exc. "+par_str)
    plt.xlabel("rev. temp")
    plt.ylabel("exc. ratio")
    plt.xscale("log")
    plt.ylim(-0.1,1.1)
    plt.show()
    plt.close()
    return
    
ratio_p_log_files = glob.glob(logfile_header+"_ratio_p_log_*.npy")
ratio_p_log_files = sorted(ratio_p_log_files)
ratio_p_log = np.empty((num_temp,par_num,0))
for ifile in range(len(ratio_p_log_files)):
    tfile = ratio_p_log_files[ifile]
    np_tmp = np.load(tfile,allow_pickle=True)
    ratio_p_log = np.append(ratio_p_log,np_tmp,axis=2)
sum_p_log = np.sum(ratio_p_log[:,:,burn_in_length:],axis=2)
for ipar in range(par_num):
    ratio = sum_p_log[:,ipar]/(cycle-burn_in_length)
    plt_p_ratio(ipar,temp,ratio)





# レプリカ交換率 vs 逆温度
def plt_exc_rep_log(temp,ratio):
    plt.figure()
    plt.hlines(1.0,min(temp[1:-1]),max(temp[1:-1]),colors="gray")
    plt.hlines(0.0,min(temp[1:-1]),max(temp[1:-1]),colors="gray")
    # plt.plot(temp[1:-1],sum_exc_rep[1:-1]/((cycle-burn_in_length)/2))
    # plt.scatter(temp[1:-1],sum_exc_rep[1:-1]/((cycle-burn_in_length)/2),s=3)
    plt.plot(temp[1:-1],ratio[1:-1])
    plt.scatter(temp[1:-1],ratio[1:-1],s=3)
    min_rep = np.argmin(ratio[1:-1]) + 1
    tmp_txt = "min_rep = "+str(min_rep)
    plt.text(temp[min_rep],np.min(ratio[1:-1])-0.05,tmp_txt,horizontalalignment="center",fontsize=10)
    plt.title("exc_rep_ratio")
    plt.xlabel("rev. temp")
    plt.ylabel("ratio temp")
    plt.xscale("log")
    plt.ylim(-0.1,1.1)
    return
    
ratio_exc_rep_files =  glob.glob(logfile_header+"*ratio_exc_rep_log*.npy")
ratio_exc_rep_files = sorted(ratio_exc_rep_files)
ratio_exc_rep_log = np.empty((num_temp,0))
for ifile in range(len(ratio_exc_rep_files)):
    tfile = ratio_exc_rep_files[ifile]
    np_tmp = np.load(tfile,allow_pickle=True)
    ratio_exc_rep_log = np.append(ratio_exc_rep_log,np_tmp,axis=1)        
sum_exc_rep = np.sum(ratio_exc_rep_log[:,burn_in_length:],axis=1)
ratio = sum_exc_rep/((cycle-burn_in_length)/2)
plt_exc_rep_log(temp,ratio)





### 08.fittingパラメータのBayes推定
+ Bayes自由エネルギーの計算
+ 最適な逆温度の決定
+ データのノイズ推定
+ MAP推定


In [ ]:
from jnbayes.libs.bayes_sub import calc_free_energy
from jnbayes.libs.bayes_sub import get_each_Y


####### calc free energy
### Bayes自由エネルギー計算
num_par_1000 = int((cycle-burn_in_length)/log_length)
sum_E_log = np.array([np.sum(E_log[:,burn_in_length+i*log_length:burn_in_length+(i+1)*log_length],axis=1) for i in range(num_par_1000)])
mean_E_log = sum_E_log / log_length
E_free = calc_free_energy(E_log,mean_E_log,temp,num_par_1000,data_len,logfile_header)
E_free_ave = np.mean(E_free,axis=0)
mean_E_log_ave = np.mean(E_free,axis=0)

######## 逆温度推定
iE_min_rep = np.argmin(E_free[:,1:-1],axis=-1)+1
iE_min_rep_ave = np.argmin(E_free_ave[1:-1],axis=-1)+1
Ef_min = np.min(E_free[:,1:-1],axis=-1)
Ef_min_ave = np.min(mean_E_log_ave[1:-1],axis=-1)

######## ノイズ推定
noise_result = 1/(temp[iE_min_rep_ave])**0.5

print("最適な逆温度のインデックス",iE_min_rep_ave)
print("自由エネルギーの最小値",Ef_min_ave)
print("推定されたノイズの標準偏差",noise_result)


########　MAP
iE_min_cycle = np.argmin(E_log[iE_min_rep_ave])
map_par = p_log[iE_min_rep_ave,:,iE_min_cycle]

Y_dic = get_each_Y(X,func_dict,map_par)
print(Y_dic)







### 09.結果の表示(08の)
+ Bayes自由エネルギー vs 逆温度
+ 最適な事後確率分布

In [ ]:
from jnbayes.libs.etc_sub import COLOER_MAP


### Bayes自由エネルギー vs 逆温度
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.xlabel("rev. temp")
plt.xscale("log")
plt.ylabel("Efree")
plt.title("Efree vs rev. temp")
plt.plot(temp[1:-1],E_free_ave[1:-1])

plt.subplot(1,2,2)
plt.xlabel("rev. temp")
plt.xscale("log")
plt.ylabel("Efree")
plt.title("<-- zoom")
plt.plot(temp[1:-1],E_free_ave[1:-1])
Y_LIM_LOW = Ef_min_ave - 1
Y_LIM_HIGH = Ef_min_ave + 7
X_LIM_LOW_INDEX = int(iE_min_rep_ave - 4)
X_LIM_HIGH_INDEX = int(iE_min_rep_ave + 5)
plt.ylim(Y_LIM_LOW, Y_LIM_HIGH)
plt.xlim(temp[X_LIM_LOW_INDEX], temp[X_LIM_HIGH_INDEX])

plt.show()
plt.close()


### 最適な事後確率分布
for ipar in range(len(p_log[0,:,0])):
    tmp_p_log = p_log[iE_min_rep_ave][ipar][burn_in_length:]
    plt.title(f"posterior probability(rep={iE_min_rep_ave}, par={ipar})")
    plt.hist(tmp_p_log,bins=40)
    plt.xlabel(f"par{ipar}")
    plt.ylabel("Cnt")
    plt.show()
    plt.close()
